In [37]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

session = sagemaker.Session()
role = get_execution_role()
region_name = boto3.Session().region_name

In [38]:
%store -r s3_bucket
%store -r prefix

In [39]:
s3_train_data = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "train"
)
s3_validation_data = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "validation"
)
s3_train_lst_path = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "train_lst"
)
s3_validation_lst_path = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "validation_lst"
)
s3_output_location = 's3://{}/{}/output'.format(
    s3_bucket, 
    prefix
)

In [40]:
from sagemaker.image_uris import retrieve 

container = retrieve(
    "image-classification", 
    region_name, 
    "1"
)
container

'811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1'

In [41]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.p2.xlarge',
    output_path=s3_output_location,
    sagemaker_session=session
)

In [42]:
estimator.set_hyperparameters(
    num_layers=18,
    image_shape = "1,28,28",
    num_classes=10,
    num_training_samples=600,
    mini_batch_size=20,
    epochs=5,
    learning_rate=0.01,
    top_k=2,
    precision_dtype='float32'
)

In [43]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(
    s3_train_data, 
    distribution='FullyReplicated', 
    content_type='application/x-image', 
    s3_data_type='S3Prefix'
)

validation = TrainingInput(
    s3_validation_data, 
    distribution='FullyReplicated', 
    content_type='application/x-image', 
    s3_data_type='S3Prefix'
)

In [44]:
content_type = 'application/x-image'

train_lst = TrainingInput(
    s3_train_lst_path, 
    distribution='FullyReplicated', 
    content_type=content_type, 
    s3_data_type='S3Prefix'
)

validation_lst = TrainingInput(
    s3_validation_lst_path, 
    distribution='FullyReplicated', 
    content_type=content_type, 
    s3_data_type='S3Prefix'
)

In [45]:
data_channels = {
    'train': train, 
    'validation': validation,
    'train_lst': train_lst,
    'validation_lst': validation_lst
}

In [46]:
%%time

estimator.fit(inputs=data_channels, logs=True)

2021-06-01 10:40:12 Starting - Starting the training job...
2021-06-01 10:40:36 Starting - Launching requested ML instancesProfilerReport-1622544012: InProgress
......
2021-06-01 10:41:39 Starting - Preparing the instances for training............
2021-06-01 10:43:37 Downloading - Downloading input data.........
2021-06-01 10:45:06 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[06/01/2021 10:45:12 INFO 140389266315072] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[06/01/2021 10:45:12 INFO 140389266315072] Merging with provided configuration f

In [47]:
endpoint = estimator.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge'
)

-----------------!

In [48]:
from sagemaker.serializers import IdentitySerializer

endpoint.serializer = IdentitySerializer(
    content_type="application/x-image"
)

In [49]:
import json

def get_class_from_results(results):
    results_prob_list = json.loads(results)
    best_index = results_prob_list.index(
        max(results_prob_list)
    )
    
    return {
        0: "ZERO",
        1: "ONE",
        2: "TWO",
        3: "THREE",
        4: "FOUR",
        5: "FIVE",
        6: "SIX",
        7: "SEVEN",
        8: "EIGHT",
        9: "NINE"
    }[best_index]

In [50]:
from IPython.display import Image, display

def predict(filename, endpoint=endpoint):
    byte_array_input = None
    
    with open(filename, 'rb') as image:
        f = image.read()
        byte_array_input = bytearray(f)
        
    display(Image(filename))
        
    results = endpoint.predict(byte_array_input)
    return get_class_from_results(results)

In [51]:
!ls tmp/test

ABNIQFCOZDYW.png  FBCKNYWRMVAL.png  MCEUSTQLPRGN.png  SAHXCBUPZLOG.png
AEMFUYPHGNSX.png  FCZLDWIMUNYS.png  MEHCRTJPVNLO.png  SAKZHMJGREOD.png
AHVCEJDLQZOR.png  FLDUKBHGTWOX.png  MFJYTRNCEBWU.png  SANKQJHVGFRL.png
AHWQKVLBCUOR.png  FLJMOYXRHIKW.png  MHQBWUEZGAYN.png  SFXKLJZURTIA.png
AJQDBWKPUMFG.png  FTMAZXUOLYCQ.png  MJRIBGESPVAZ.png  SLQTEXVMGHIZ.png
AJQESTBVOMFI.png  GELDKRMQNJIS.png  MSLBWXHRKAUY.png  STGMIQFHENDV.png
AJQOXEGHPRBU.png  GFDMQYTLVWEK.png  MZONWIRLAVCB.png  SVYNKAXGMUPT.png
ALHPGYRCUIDW.png  GLRHIUBZSNOJ.png  NBJCXRWOHSTF.png  SYBETGMCDQKV.png
ALIOSDZTPXUB.png  GPFXSOLKEJTD.png  NCJBTZKWGOXQ.png  SZBUCWYHMFIE.png
APJWXCGOLTQF.png  GSCXQVALUDNM.png  NIBTWMLUAXGQ.png  TJMUPOQISDRK.png
APLVTIEZMRQS.png  GSNVFPOIRCKY.png  NMAPFVJGLIQK.png  TOGRKZJUNQFA.png
ASGCLZFUKMYQ.png  GSZWRVNHFJEC.png  NQIYVXOFJUDG.png  TZYCKDNSLFPH.png
BALOJKEPMCYG.png  GVWTPHINKSQO.png  NQJBFOSLEZAK.png  UCAZEPQBVRDL.png
BEOCULSWTHYQ.png  GWECYAUFIPVO.png  OBDPUEQFNCKS.png  UTWFXCKIMVZQ.png
BEQYLR

In [52]:
results = !ls -1 tmp/test
for filename in results:
    print(predict(f"tmp/test/{filename}"))

ZERO


SEVEN


ZERO


NINE


NINE


...

TWO


ONE


TWO


ONE


FOUR


In [53]:
endpoint.delete_endpoint()

In [54]:
tn = estimator.latest_training_job.name
training_job_name = tn
%store training_job_name

Stored 'training_job_name' (str)
